# Library import

In [1]:
#Import de librerias basicas tablas y matrices
import numpy as np 
import pandas as pd 

#Gradient Boosting
import lightgbm as lgb

#Funciones auxiliares sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold #Split y cross Validation
from sklearn.metrics import cohen_kappa_score, accuracy_score, balanced_accuracy_score #Metricas
from sklearn.utils import shuffle 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn import set_config



#Visualizacióon
from plotly import express as px

#Plot de matriz de confusion normalizada en actuals
import sys
sys.path.append('../Scripts')
from utils import plot_confusion_matrix

import os

#Optimizacion de hiperparametros
import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

#Guardado de objetos en archivos joblib
from joblib import load, dump

/home/ge/MCD/LaboII/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Env

In [2]:
# Paths para acceso archivos
#Este notebook asume la siguiente estructura de carpetas a partir de la ubicacion de base_dir 
#(dos niveles arriba de la carpeta donde se ejecuta el notebook). 
# /UA_MDM_LDI_II/
# /UA_MDM_LDI_II/input
# /UA_MDM_LDI_II/input/petfinder-adoption-prediction/            <- Aca deben ir todos los archivos de datos de la competencia 
# /UA_MDM_LDI_II/tutoriales/                       <- Aca deben poner los notebooks y scripts que les compartimos
# /UA_MDM_LDI_II/work/                             <- Resultados de notebooks iran dentro de esta carpeta en subcarpetas
# /UA_MDM_LDI_II/work/models/                     <- Modelos entrenados en archivos joblibs
# /UA_MDM_LDI_II/work/optuna_temp_artifacts/      <- Archivos que queremos dejar como artefacto de un trial de optuna (optuna los copiara a la carpeta de abajo)
# /UA_MDM_LDI_II/work/optuna_artifacts/           <- Archivos con artefactos que sibimos a optuna

#Subimos dos niveles para quedar en la carpeta que contiene input y UA_MDM_LDI_II
BASE_DIR = '../'

#Datos de entrenamiento 
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")
PATH_TO_TEST = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/test/test.csv")

#Salida de modelos entrenados
PATH_TO_MODELS = os.path.join(BASE_DIR, "work/models")

#Artefactos a subir a optuna
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")

#Artefactos que optuna gestiona
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")


SEED = 42 #Semilla de procesos aleatorios (para poder replicar exactamente al volver a correr un modelo)
TEST_SIZE = 0.2 #Facción para train/test= split

# Config for pandas output from pipelines
set_config(transform_output = "pandas")

In [3]:
# Datos Tabulares
df_train = pd.read_csv(PATH_TO_TRAIN)
df_train.shape

(14993, 24)

In [4]:
# Datos Tabulares
df_test = pd.read_csv(PATH_TO_TEST)
df_test.shape

(3972, 23)

In [5]:
# Otros archivos

# States diccionary
df_state_labels = pd.read_csv(os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/StateLabels.csv"))
df_state_labels['State_Pop'] = [
    3794,
    2194,
    1929,
    1746,
    100,
    937,
    1129,
    1685,
    2509,
    255,
    1774,
    3833,
    2822,
    6555,
    1275
]

### Categorical variables


| Variable       | Type         | Description                                                                           |
|----------------|--------------|---------------------------------------------------------------------------------------|
| `PetID`        | Categorical  | ID (Should be dropped)                                                                |
| `AdoptionSpeed`| Categorical  | Target variable                                                                       |
| `Type`         | Categorical  | 1 = Cat, 2 = Dog                                                                      |
| `Name`         | Categorical  | Name of pet                                                                           |
| `Breed1`       | Categorical  | See BreedLabels dictionary                                                            |
| `Breed2`       | Categorical  | See BreedLabels dictionary                                                            |
| `Gender`       | Categorical  | 1 = Male, 2 = Female, 3 = Mixed (used for groups)                                     |
| `Color1`       | Categorical  | See ColorLabels dictionary                                                            |
| `Color2`       | Categorical  | See ColorLabels dictionary                                                            |
| `Color3`       | Categorical  | See ColorLabels dictionary                                                            |
| `MaturitySize` | Categorical  | 1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified                  |
| `FurLength`    | Categorical  | 1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified                                    |
| `Vaccinated`   | Categorical  | 1 = Yes, 2 = No, 3 = Not Sure                                                         |
| `Dewormed`     | Categorical  | 1 = Yes, 2 = No, 3 = Not Sure                                                         |
| `Sterilized`   | Categorical  | 1 = Yes, 2 = No, 3 = Not Sure                                                         |
| `Health`       | Categorical  | 1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified                  |
| `State`        | Categorical  | See StateLabels dictionary                                                            |
| `RescuerID`    | Categorical  | ID                                                                                    |
| `Description`  | Text         | Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese. |

### Quantitative Variables

| Variable   | Type             |Description                                           |
|------------|------------------|------------------------------------------------------|
| `Age`      |  Numerical       |Age of pet when listed, in months                     |
| `Quantity` |  Numerical       |Number of pets represented in profile                 |
| `Fee`      |  Numerical       |Adoption fee (0 = Free)                               |
| `VideoAmt` |  Numerical       |Total uploaded videos for this pet                    |
| `PhotoAmt` |  Numerical       |Total uploaded photos for this pet                    |

In [6]:
!which python

/home/ge/MCD/LaboII/.venv/bin/python


# FE

In [7]:

def apply_fe(dataset):
    # Name
    # Feature to know if the pet has a name
    unknown_names = 'name|puppies|kitten|puppy|unknown'
    dataset['Name'] = np.where(dataset['Name'].str.lower().str.contains(unknown_names), np.nan, dataset['Name'])
    dataset['Name'] = np.where(dataset['Name'].str.len() < 3, np.nan, dataset['Name'])
    dataset['HasName'] = dataset['Name'].apply(lambda x: 0 if pd.isnull(x) else 1)

    
    # Breed
    # Unify values in Breed1
    dataset['Breed1'] = np.where((dataset['Breed1']==0) & (dataset['Breed2']!=0), dataset['Breed2'], dataset['Breed1'])
    dataset['Breed2'] = np.where((dataset['Breed1']==dataset['Breed2']), 0, dataset['Breed2'])
    
    # Merge Breed1 and Breed2
    dataset['FullBreed'] = dataset['Breed1'].astype(str) + '_' + dataset['Breed2'].astype(str) 
    # Pure breed
    dataset['PureBreed'] = np.where((dataset['Breed2'] == 0) & (~dataset['Breed1'].isin([307, 266, 265, 264])), 1, 0)
    
    # Color
    # Merge Color1, Color2 and Color3
    dataset['Color'] = dataset['Color1'].astype(str) + '_' + dataset['Color2'].astype(str) + '_' + dataset['Color3'].astype(str)
    # Monochromatic
    dataset['Monochromatic'] = np.where((dataset['Color2'] == 0) & (dataset['Color3'] == 0), 1, 0)
    
    # Health
    # Merge Vaccinated, Dewormed and Sterilized to know if the pet is up to date with routine stuff
    dataset['Va_De_St'] = dataset['Vaccinated'].astype(str) + '_' + dataset['Dewormed'].astype(str) + '_' + dataset['Sterilized'].astype(str)

    # Code to add a min age for sterilization.... But EDA shows that they do it anyway
    #df['CanBeSter'] = np.where((df['Age'] > 9) & (df['Type'] == 1) | (df['Age'] > 6) & (df['Type'] == 2), 1, 0)
    
    # Fee
    # Create fee bins using log transformation
    dataset['Fee_bins'] = pd.cut(np.log1p(dataset.Fee), 5, labels=['Fee_{}'.format(e) for e in range(5)])

    #RescuerID
    rescuer_count = dataset.groupby(['RescuerID'])['PetID'].count().reset_index()
    rescuer_count.columns = ['RescuerID', 'Rescuer_count']
    dataset = dataset.merge(rescuer_count, how='left', on='RescuerID')

    # Age
    dataset['RelAge'] = np.where(dataset['Type'] == 1, dataset['Age']/144, dataset['Age']/180) # Vida media de un gato 12 años, de un perro 15

    # Multimedia
    dataset['Total_photo_video'] = dataset['PhotoAmt'] + dataset['VideoAmt']

    # Si tiene todo lo del veterinatrio en orden
    dataset['VetInOrder'] = np.where((dataset['Vaccinated'] == 1) & (dataset['Dewormed'] == 1) & (dataset['Sterilized'] == 1) & (dataset['Health'] == 1), 1, 0)

    # Si le falta info sobre lo relacionado a veterinario
    dataset['NoVet'] = np.where((dataset['Vaccinated'] == 3) | (dataset['Dewormed'] == 3) | (dataset['Sterilized'] == 3) | (dataset['Health'] == 3), 1, 0)

    # Longitud de la descripcion
    dataset['LenDesc'] = dataset['Description'].apply(lambda x: len(str(x)))

    # variable "age_categ" para categorizar age (identica para perros y gatos, sin problemas según fuentes)
    age_categ_cuts = [
        (dataset['Age'].le(1*12)),
        (dataset['Age'].gt(1*12) & dataset['Age'].le(3*12)),
        (dataset['Age'].gt(3*12) & dataset['Age'].le(9*12)),
        (dataset['Age'].gt(9*12))   
    ]
    age_categ_values = ["baby","young","adult","old"]
    dataset['Age_categ'] = np.select(age_categ_cuts, age_categ_values)

    # variable "individual" para saber si es un animal solo o un grupo
    dataset['Individual'] = np.where((dataset['Quantity'].gt(1)),1,0)

    #variable "free" para saber si hay que pagar o no por adoptar
    dataset['Free'] = np.where((dataset['Fee'].gt(0)),1,0)

    # agrego state label y state population
    dataset = pd.merge(dataset, df_state_labels, how = "left", left_on='State',right_on='StateID')

    # Return the augmented dataset
    return dataset

In [8]:
df_train = apply_fe(df_train.copy())
df_test = apply_fe(df_test.copy())

In [9]:
y = df_train['AdoptionSpeed']
X = df_train.drop(columns=['AdoptionSpeed'])

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                    test_size=TEST_SIZE, 
                                                    random_state=SEED, 
                                                    stratify=y)

In [11]:
# Calculate the mean adoption speed per state and add that value to train and test datasets
state_mean = pd.concat([X_train['State'], y_train], axis=1).groupby('State')['AdoptionSpeed'].mean().reset_index()
state_mean.columns = ['State', 'AdoptionSpeed_mean']
X_train = X_train.merge(state_mean, how='left', on='State')
X_val = X_val.merge(state_mean, how='left', on='State')

In [12]:
X_train.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'HasName', 'FullBreed',
       'PureBreed', 'Color', 'Monochromatic', 'Va_De_St', 'Fee_bins',
       'Rescuer_count', 'RelAge', 'Total_photo_video', 'VetInOrder', 'NoVet',
       'LenDesc', 'Age_categ', 'Individual', 'Free', 'StateID', 'StateName',
       'State_Pop', 'AdoptionSpeed_mean'],
      dtype='object')

In [13]:
X_train.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'HasName', 'FullBreed',
       'PureBreed', 'Color', 'Monochromatic', 'Va_De_St', 'Fee_bins',
       'Rescuer_count', 'RelAge', 'Total_photo_video', 'VetInOrder', 'NoVet',
       'LenDesc', 'Age_categ', 'Individual', 'Free', 'StateID', 'StateName',
       'State_Pop', 'AdoptionSpeed_mean'],
      dtype='object')

In [14]:
#Armo listas con features de texto y numericas
char_feats = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized','Health', 'State', 'RescuerID',
       'Description', 'HasName', 'FullBreed',
       'PureBreed', 'Color', 'Monochromatic', 'Va_De_St', 'Fee_bins', 'VetInOrder', 'NoVet','VideoAmt','PhotoAmt','Total_photo_video', 'Name',
       'Age_categ', 'Individual', 'Free', 'StateName'
       ]
#
numeric_feats = ['Age', 'Quantity', 'Fee', 'RelAge',
       'Rescuer_count', 'LenDesc', 'AdoptionSpeed_mean', ]

fe_drop = ['PetID','Description']

In [15]:
# Para revisar que tenemos todas las columnas en las variables
X_train.shape[1]-len(fe_drop)-len(char_feats)-len(numeric_feats)

1

In [16]:
X_train_id = X_train['PetID']
X_train = X_train[char_feats + numeric_feats]

X_val_id = X_val['PetID']

#X_val_id.to_csv(os.path.join(BASE_DIR, "input/val_id.csv'), index=False)
X_val = X_val[char_feats + numeric_feats]

In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(min_frequency= 30, handle_unknown= 'use_encoded_value', unknown_value= -1), char_feats)],
        remainder= 'passthrough')

In [18]:
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)

In [19]:
#Entreno un modelo inicial sin modificar hiperparametros. Solamente especifico el numero de clases y el tipo de modelo como clasificacoión
lgb_params = params = {
                        'objective': 'multiclassova',
                        'num_class': 5,
                        'learning_rate': 0.01,
                        'n_estimators': 1000,
                        'device': 'cpu'
                        }


#genero el objeto Dataset que debo pasarle a lightgbm para que entrene
lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)

#entreno el modelo con los parametros por defecto
lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

[LightGBM] [Info] Number of positive: 328, number of negative: 11666
[LightGBM] [Info] Number of positive: 2472, number of negative: 9522
[LightGBM] [Info] Number of positive: 3230, number of negative: 8764
[LightGBM] [Info] Number of positive: 2607, number of negative: 9387
[LightGBM] [Info] Number of positive: 3357, number of negative: 8637
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000928 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 11994, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.027347 -> initscore=-3.571420
[LightGBM] [Info] Start training from score -3.571420
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.206103 -> initscore=-1.348577
[LightGBM] [Info] Start training from score -1.348577
[LightGBM] [Info] [binary:BoostFrom

In [20]:
y_pred = lgb_model.predict(X_val).argmax(axis=1)

#Calculo el Kappa
cohen_kappa_score(y_val,y_pred, weights = 'quadratic')

0.3483360166031384

In [21]:
feature_importance = pd.DataFrame({'feature': preprocessor.get_feature_names_out(), 'weight': lgb_model.feature_importance(importance_type='gain')})
feature_importance.sort_values('weight', ascending=False).head(10)

,feature,weight
38,remainder__LenDesc,117271.395867
37,remainder__Rescuer_count,115353.877126
36,remainder__RelAge,85741.076378
1,cat__Breed1,50646.854816
26,cat__PhotoAmt,47276.312706
19,cat__Color,42568.556228
33,remainder__Age,38724.610719
17,cat__FullBreed,29661.240513
21,cat__Va_De_St,27408.956937
39,remainder__AdoptionSpeed_mean,24945.529847


In [22]:
#Genero una metrica para que lightGBM haga la evaluación y pueda hacer early_stopping en el cross validation
def lgb_custom_metric_kappa(dy_pred, dy_true):
    metric_name = 'kappa'
    value = cohen_kappa_score(dy_true.get_label(),dy_pred.argmax(axis=1),weights = 'quadratic')
    is_higher_better = True
    return(metric_name, value, is_higher_better)

#Funcion objetivo a optimizar. En este caso vamos a hacer 5fold cv sobre el conjunto de train. 
# El score de CV es el objetivo a optimizar. Ademas vamos a usar los 5 modelos del CV para estimar el conjunto de test,
# registraremos en optuna las predicciones, matriz de confusion y el score en test.
# CV Score -> Se usa para determinar el rendimiento de los hiperparametros con precision 
# Test Score -> Nos permite testear que esta todo OK, no use (ni debo usar) esos datos para nada en el entrenamiento 
# o la optimizacion de hiperparametros

def cv_es_lgb_objective(trial):

    #PArametros para LightGBM
    lgb_params = {      
                        #PArametros fijos
                        'objective': 'multiclassova',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        'seed': SEED,
                        
                        #Hiperparametros a optimizar utilizando suggest_float o suggest_int segun el tipo de dato
                        #Se indica el nombre del parametro, valor minimo, valor maximo 
                        #en elgunos casos el parametro log=True para parametros que requieren buscar en esa escala
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-7, 1e-4, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-5, 0.1, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 100, 256),
                        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.01),
                        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 15, 70),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.9),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 3),
                        'min_child_samples': trial.suggest_int('min_child_samples', 20, 60),
                        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
                        'min_split_gain': trial.suggest_float('min_split_gain', 5, 8),
                        "max_bin": trial.suggest_int('max_bin', 300, 500),
                        } 

    #Voy a generar estimaciones de los 5 modelos del CV sobre los datos test y los acumulo en la matriz scores_ensemble
    scores_ensemble = np.zeros((len(y_val),len(y_train.unique())))

    #Score del 5 fold CV inicializado en 0
    score_folds = 0

    #Numero de splits del CV
    n_splits = 5

    #Objeto para hacer el split estratificado de CV
    skf = StratifiedKFold(n_splits=n_splits)

    for i, (if_index, oof_index) in enumerate(skf.split(X_train, y_train)):
        
        #Dataset in fold (donde entreno) 
        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                        label=y_train.iloc[if_index],
                                        free_raw_data=False)
        
        #Dataset Out of fold (donde mido la performance del CV)
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                        label=y_train.iloc[oof_index],
                                        free_raw_data=False)

        #Entreno el modelo
        lgb_model = lgb.train(lgb_params,
                                lgb_if_dataset,
                                valid_sets=lgb_oof_dataset,
                                feval = lgb_custom_metric_kappa
                                )
        
        #Acumulo los scores (probabilidades) de cada clase para cada uno de los modelos que determino en los folds
        #Se predice el 20% de los datos que separe para tes y no uso para entrenar en ningun fold
        scores_ensemble = scores_ensemble + lgb_model.predict(X_val)
        
        #Score del fold (registros de dataset train que en este fold quedan out of fold)
        score_folds = score_folds + cohen_kappa_score(y_train.iloc[oof_index], 
                                                            lgb_model.predict(X_train.iloc[oof_index]).argmax(axis=1),weights = 'quadratic')/n_splits


    #Guardo prediccion del trial sobre el conjunto de test
    # Genero nombre de archivo
    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
    # Copia del dataset para guardar la prediccion
    predicted_df = pd.DataFrame(X_val,y_val).copy()
    # Genero columna pred con predicciones sumadas de los 5 folds
    predicted_df['pred'] = [scores_ensemble[p,:] for p in range(scores_ensemble.shape[0])]
    # Grabo dataframe en temp_artifacts
    dump(predicted_df, predicted_filename)
    # Indico a optuna que asocie el archivo generado al trial
    #upload_artifact(trial, predicted_filename, artifact_store)    

    #Grabo natriz de confusion
    #Nombre de archivo
    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
    #Grabo archivo
    plot_confusion_matrix(y_val,scores_ensemble.argmax(axis=1)).write_image(cm_filename)
    #Asocio al trial
    #upload_artifact(trial, cm_filename, artifact_store)

    #Determino score en conjunto de test y asocio como metrica adicional en optuna
    test_score = cohen_kappa_score(y_val,scores_ensemble.argmax(axis=1),weights = 'quadratic')
    trial.set_user_attr("test_score", test_score)

    #Devuelvo score del 5fold cv a optuna para que optimice en base a eso
    return(score_folds)

In [24]:
#Inicio el store de artefactos (archivos) de optuna
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

#Genero estudio
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name="Final - Ge - LGB Multiclass CV",
                            load_if_exists = True)
#Corro la optimizacion
run_optuna = True
optuna.logging.set_verbosity(optuna.logging.WARNING)
if run_optuna:
    study.optimize(cv_es_lgb_objective, n_trials=200, show_progress_bar = True)

Best trial: 52. Best value: 0.361781:  50%|█████     | 100/200 [1:42:14<1:42:14, 61.35s/it]


[W 2024-10-13 16:04:03,811] Trial 100 failed with parameters: {'lambda_l1': 1.2578274755080858e-05, 'lambda_l2': 0.0010561116496413025, 'num_leaves': 237, 'learning_rate': 0.008602492972600115, 'min_data_in_leaf': 25, 'feature_fraction': 0.6086483948711785, 'bagging_fraction': 0.7303466623417876, 'bagging_freq': 2, 'min_child_samples': 20, 'n_estimators': 1219, 'min_split_gain': 5.0501879969712204, 'max_bin': 420} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/ge/MCD/LaboII/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_1936/3634229298.py", line 67, in cv_es_lgb_objective
    lgb_model = lgb.train(lgb_params,
                ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ge/MCD/LaboII/.venv/lib/python3.12/site-packages/lightgbm/engine.py", line 368, in train
    evaluation_result_list.extend(booster.eval_valid(fev

KeyboardInterrupt: 

In [26]:
sorted_trials = sorted(study.get_trials(), key=lambda trial: trial.value, reverse=True)
for trial in sorted_trials:
    print(f"Trial {trial.number}: {trial.params}")

TypeError: '<' not supported between instances of 'float' and 'NoneType'

In [23]:
lgbm_params = {'lambda_l1': 0.07958720818703245, 
'lambda_l2': 0.23359973422583905, 
'num_leaves': 179, 
'min_data_in_leaf': 28, 
'feature_fraction': 0.48667927053386195, 
'bagging_fraction': 0.9896417154375905, 
'bagging_freq': 5, 
'min_child_samples': 58, 
'min_gain_to_split': 0.5653596125981446,
'seed': SEED}

In [24]:
#!optuna-dashboard sqlite:///../work/db1.sqlite3 --artifact-dir ../work/optuna_artifacts --port 8081

In [ ]:
#Vamos a replicar el resultado de la optimizacion reentrenando el modelo con el mejor conjunto de hiperparametros
#Generamos parametros incluyendo los fijos y la mejor solución que encontro optuna
lgb_params =  {      **lgbm_params,
                        'objective': 'multiclassova',
                        'verbosity':-1,
                        'num_class': len(y_train.unique())}

lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


#Entreno
lgb_model = lgb.train(lgb_params,
                    lgb_train_dataset)

lgb_proba = lgb_model.predict(X_val)
lgb_pred = lgb_model.predict(X_val).argmax(axis=1)

#Muestro matriz de confusion y kappa
display(plot_confusion_matrix(y_val, lgb_pred))

cohen_kappa_score(y_val,lgb_pred,
                  weights = 'quadratic')


In [ ]:
feature_importance = pd.DataFrame({'feature': preprocessor.get_feature_names_out(), 'weight': lgb_model.feature_importance(importance_type='gain')})
feature_importance.sort_values('weight', ascending=False).head(20)

## Blend

In [54]:
nn_proba = load(os.path.join(BASE_DIR, "resultados_nn_bert/test_04 ResNet_1.0.0_0(2).joblib"))
nn_proba = nn_proba[['PetID', 'pred']]

bert_proba = load(os.path.join(BASE_DIR, "resultados_nn_bert/test_06 Bert_1.0_0.joblib"))
bert_proba = bert_proba[['PetID', 'pred']]

In [55]:
lgb_proba_df = pd.DataFrame({'PetID': X_val_id,
                            'lgbm_proba': list(lgb_proba)})

all_proba = lgb_proba_df.merge(bert_proba[['PetID', 'pred']].rename({'pred':'bert_proba'},axis=1),
                  on='PetID', how='left').merge(nn_proba[['PetID', 'pred']].rename({'pred':'nn_proba'},axis=1),
                  on='PetID', how='left').merge(df_train[['PetID','AdoptionSpeed']], on='PetID', how='left')

all_proba['bert_proba'] = [np.zeros(5) if type(i) is float else  i for i in all_proba['bert_proba']]
all_proba['nn_proba'] = [np.zeros(5) if type(i) is float else  i for i in all_proba['nn_proba']]

In [56]:
y = all_proba['AdoptionSpeed']

In [ ]:
# Calculate the quadratic weighted kappa
lgbm_kappa = cohen_kappa_score(y, np.array(all_proba['lgbm_proba'].tolist()).argmax(axis=1), weights='quadratic')
# Calculate the quadratic weighted kappa
bert_kappa = cohen_kappa_score(y, np.array(all_proba['bert_proba'].tolist()).argmax(axis=1), weights='quadratic')
# Calculate the quadratic weighted kappa
nn_kappa = cohen_kappa_score(y, np.array(all_proba['nn_proba'].tolist()).argmax(axis=1), weights='quadratic')

print(f"Kappa LGBM: {lgbm_kappa}, Kappa BERT: {bert_kappa}, Kappa NN: {nn_kappa}")


In [ ]:
def objective(trial):
    # Suggest weights for each model's predictions
    w_lgbm = trial.suggest_float('w_lgbm', 0.5, 1.0)
    w_bert = trial.suggest_float('w_bert', 0.0, 0.5)
    w_nn = trial.suggest_float('w_nn', 0.0, 0.5)

    # Normalize weights
    w_sum = w_lgbm + w_bert + w_nn
    w_lgbm /= w_sum
    w_bert /= w_sum
    w_nn /= w_sum

    # Calculate the blended predictions
    y_pred = np.array((w_lgbm* all_proba['lgbm_proba']+ 
                        w_bert * all_proba['bert_proba']+
                        w_nn* all_proba['nn_proba']).tolist()).argmax(axis=1)
    
    # Calculate the quadratic weighted kappa
    kappa = cohen_kappa_score(y, y_pred, weights='quadratic')

    return kappa

# Create an Optuna study
optuna.logging.set_verbosity(optuna.logging.WARNING)
#optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, show_progress_bar = True)

# Best weights found
best_weights = study.best_trial.params
print("Best weights:", best_weights)

In [ ]:
print(f"Kappa blend {study.best_value}, \nKappa LGBM: {lgbm_kappa}, \nKappa BERT: {bert_kappa}, \nKappa NN: {nn_kappa}")